##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [5]:
import os
# from kaggle_secrets import UserSecretsClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

try:
    os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Google API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Google API key setup complete.


In [4]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 5.1 Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [ ]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [ ]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The `LoopAgent` itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

1. A simple Python function that the `LoopAgent` understands as an "exit" signal.
2. An agent that can call that function when the right condition is met.

First, you'll define the `exit_loop` function:

In [21]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


To let an agent call this Python function, we wrap it in a `FunctionTool`. Then, we create a `RefinerAgent` that has this tool.

👉 **Notice its instructions:** this agent is the "brain" of the loop. It reads the `{critique}` from the `CriticAgent` and decides whether to (1) call the `exit_loop` tool or (2) rewrite the story.

In [ ]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    
    output_key="current_story", # It overwrites the story with the new, refined version.
    tools=[FunctionTool(exit_loop)], # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of `max_iterations`:

In [23]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2, # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


Let's run the agent and give it a topic to write a short story about:

In [26]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/sneha/Documents/Projects/Kaggle_ai_course/myvenv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

---
### Section 7
## 🔄 MATLAB to Python Code Migration with LoopAgent

**Use Case: Code Conversion with Iterative Refinement**

Converting code from one language to another (like MATLAB to Python) is rarely perfect on the first attempt. The converted code may have:
- Incorrect library mappings
- Syntax errors
- Missing imports
- Non-Pythonic patterns

A **LoopAgent** is ideal for this task because it allows:
1. Initial conversion attempt
2. Validation and critique
3. Iterative refinement until the code meets quality standards

**Architecture: Converter → Validator → Refiner Loop**

We'll build a system with three specialized agents:
1. **Initial Converter Agent** - Converts MATLAB code to Python using pandas and matplotlib
2. **Validator Agent** - Reviews the Python code for correctness and best practices
3. **Refiner Agent** - Improves the code or signals loop exit when approved

In [13]:
# Read the MATLAB code from file
with open('MATLAB.txt', 'r') as f:
    matlab_code = f.read()

print("✅ MATLAB code loaded successfully")
print(f"Code length: {len(matlab_code)} characters")

✅ MATLAB code loaded successfully
Code length: 2161 characters


In [14]:
# Initial Converter Agent: Converts MATLAB code to Python
initial_converter_agent = Agent(
    name="InitialConverterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are an expert MATLAB to Python code converter.

The user will provide MATLAB code in their message. Convert it to Python using:
- pandas for data reading and manipulation (instead of MATLAB's readtable)
- matplotlib.pyplot for plotting (instead of MATLAB's plot functions)
- numpy for numerical operations (instead of MATLAB's array operations)

Requirements:
1. Convert ALL functionality from MATLAB to Python
2. Use pandas.read_csv() instead of readtable()
3. Use matplotlib subplots instead of tiledlayout
4. Handle NaN filtering with pandas methods
5. Preserve all plot formatting (titles, labels, legends, grid, xlim)
6. Add proper imports at the top
7. Include comments explaining key conversions
8. Make the code executable and complete

Output ONLY the Python code with no explanations before or after.""",
    output_key="python_code",
)

print("✅ initial_converter_agent created.")

✅ initial_converter_agent created.


In [15]:
# Validator Agent: Reviews the Python code for correctness
validator_agent = Agent(
    name="ValidatorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Python code reviewer specializing in MATLAB-to-Python migrations.

Review the following Python code that was converted from MATLAB:

Python Code:
{python_code}

Evaluation Criteria:
1. **Completeness**: All MATLAB functionality converted?
2. **Correctness**: Proper pandas/matplotlib/numpy usage?
3. **Imports**: All necessary libraries imported?
4. **Data handling**: NaN filtering done correctly with pandas?
5. **Plotting**: Subplots configured properly? All formatting preserved?
6. **Syntax**: No Python syntax errors?
7. **Best practices**: Pythonic code style?
8. **Executability**: Would this code run without errors?

Decision:
- If the code meets ALL criteria above and would execute correctly, respond with EXACTLY: "APPROVED"
- Otherwise, provide 2-4 specific, actionable improvements needed

Focus on critical issues first (imports, syntax, functionality).""",
    output_key="validation_feedback",
)

print("✅ validator_agent created.")

✅ validator_agent created.


In [16]:
# Exit function for the loop
def exit_conversion_loop():
    """Call this function ONLY when the validation feedback is 'APPROVED', 
    indicating the Python code is correct and complete."""
    return {
        "status": "approved", 
        "message": "Python code conversion approved. Exiting refinement loop."
    }

print("✅ exit_conversion_loop function created.")

✅ exit_conversion_loop function created.


In [17]:
# Refiner Agent: Improves the code or exits the loop
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Python code refiner for MATLAB-to-Python conversions.

Current Python Code:
{python_code}

Validation Feedback:
{validation_feedback}

Your task:
- IF the validation feedback is EXACTLY "APPROVED", call the `exit_conversion_loop` function immediately and do nothing else.
- OTHERWISE, rewrite the Python code to address ALL issues mentioned in the validation feedback.

When rewriting:
1. Fix all mentioned issues completely
2. Preserve working parts of the code
3. Ensure all imports are present
4. Maintain code completeness
5. Output ONLY the improved Python code with no explanations

Output the complete, corrected Python code.""",
    output_key="python_code",  # Overwrites with improved version
    tools=[FunctionTool(exit_conversion_loop)],
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Now we combine the agents into a workflow using **SequentialAgent** with a **LoopAgent**:

1. The **initial_converter_agent** runs once to create the first Python version
2. The **LoopAgent** contains the validator and refiner agents that run repeatedly
3. The loop continues until either:
   - The validator approves the code (and refiner calls exit function)
   - Maximum iterations are reached (set to 3 for thoroughness)

In [18]:
# Create the LoopAgent for validation and refinement
conversion_refinement_loop = LoopAgent(
    name="ConversionRefinementLoop",
    sub_agents=[validator_agent, refiner_agent],
    max_iterations=3,  # Allow up to 3 refinement cycles
)

# Create the root Sequential Agent
matlab_to_python_converter = SequentialAgent(
    name="MATLABtoPythonConverter",
    sub_agents=[initial_converter_agent, conversion_refinement_loop],
)

print("✅ MATLAB to Python conversion agent system created with LoopAgent!")

✅ MATLAB to Python conversion agent system created with LoopAgent!


### Run the MATLAB to Python Conversion

Now let's run the agent system with the MATLAB code as input. The system will:
1. Convert the MATLAB code to Python (initial conversion)
2. Validate the Python code for correctness
3. Refine the code based on feedback (up to 3 iterations)
4. Exit when approved or max iterations reached

In [33]:
# Run the conversion with the MATLAB code
# Using app_name parameter to avoid mismatch warning
converter_runner = InMemoryRunner(
    agent=matlab_to_python_converter,
    app_name="MATLABtoPythonConverter"
)

# Pass the MATLAB code directly in the user message
# The initial converter will read it from the message, not from state variables
response = await converter_runner.run_debug(
    f"Here is the MATLAB code to convert to Python:\n\n{matlab_code}"
)

print("\n" + "="*60)
print("✅ Conversion workflow completed!")
print("="*60)

# Extract and save the converted Python code from session state
output_filename = 'converted_matlab_to_python.py'

try:
    # Get the most recent session to access the clean Python code
    sessions_response = await converter_runner.session_service.list_sessions(
        user_id="default_user",
        app_name="MATLABtoPythonConverter"
    )
    
    if sessions_response.sessions and len(sessions_response.sessions) > 0:
        session = await converter_runner.session_service.get_session(
            user_id="default_user",
            session_id=sessions_response.sessions[0].id
        )
        
        # Get the clean Python code from state (not from response.text which has extra content)
        if 'python_code' in session.state:
            python_code = session.state['python_code']
            
            # Save to file
            with open(output_filename, 'w') as f:
                f.write(python_code)
            
            print(f"\n✅ Python code saved to: {output_filename}")
            print(f"📂 File location: {os.path.abspath(output_filename)}")
            print(f"📏 Code length: {len(python_code)} characters")
            
            print("\n📄 Converted Python Code Preview:")
            print("-"*60)
            print(python_code)
            print("-"*60)
        else:
            print(f"\n❌ Error: 'python_code' not found in state. Available keys: {list(session.state.keys())}")
    else:
        print("\n❌ Error: No sessions found")
        
except Exception as e:
    print(f"\n❌ Error saving file: {e}")
    import traceback
    traceback.print_exc()


App name mismatch detected. The runner is configured with app name "MATLABtoPythonConverter", but the root agent was loaded from "/Users/sneha/Documents/Projects/Kaggle_ai_course/myvenv/lib/python3.13/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Here is the MATLAB code to convert to Python:

%% Time-Series Visualization Script
%
% This script reads vehicle test data from a CSV file, filters out missing
% values, and plots time-series data for wheel speed and torque.
%
% FUNCTIONALITY:
%  - Reads a CSV file containing time, speed, and torque signals.
%  - Filters non-empty (non-NaN) data points to ensure clean plotting.
%  - Creates a tiled figure layout with two subplots:
%       (1) Front-left and front-right wheel speeds (rpm)
%       (2) Front-left and front-right wheel torques (Nm)
%  - Applies consistent time limits, labels, gridlines, and legends.
%
% INPUT:
%  - CSV file with at least the following columns:
%       Time_s, spd_rpmFL, spd_rpmFR, trq_NmFL, trq_NmFR
%
% OUTPUT:
%  - A figure displaying the time-series plots for the selected data.
%
% NOTES:
%  - Update the filename variable to match the input dataset.
%  - Adjust the x-axis limits as needed to zoom into a

ValidatorAgent > I have reviewed the provided Python code against the MATLAB source and the specified conversion requirements.

The Python code successfully replicates the functionality of the MATLAB script. It uses `pandas` for data reading and manipulation, and `matplotlib` for plotting, which are appropriate Python equivalents.

Here's a breakdown of how the Python code addresses the MATLAB features:

*   **`readtable`**: Replaced with `pandas.read_csv`.
*   **`isnan`**: Replaced with `pandas.isnull()` for checking missing values.
*   **Indexing**: Boolean indexing on pandas DataFrames is used correctly.
*   **`figure` and `tiledlayout`**: Replaced with `matplotlib.pyplot.subplots`.
*   **`title`, `xlabel`, `ylabel`**: Mapped to `fig.suptitle`, `fig.supxlabel`, and `ax.set_ylabel` respectively.
*   **`nexttile`**: Implicitly handled by accessing the correct axes object from the `subplots` output (`ax[0]`, `ax[1]`).
*   **`hold all`/`hold on`**: Matplotlib's plotting functions inhere

ValidatorAgent > I have reviewed the provided Python code, which was converted from MATLAB.

Here's an evaluation based on the criteria:

1.  **Completeness**: All MATLAB functionality appears to be converted. Features like data reading, NaN filtering, time vector extraction, figure creation with tiled layouts, plotting of multiple lines on subplots, setting titles, labels, axis limits, and legends are all addressed.
2.  **Correctness**: The use of `pandas.read_csv`, `pandas.isnull()`, `matplotlib.pyplot.subplots`, and `ax.plot()` is correct for the intended operations. Boolean indexing on pandas DataFrames is also correctly implemented.
3.  **Imports**: `pandas`, `matplotlib.pyplot`, and `numpy` are imported, which are appropriate for the tasks. `numpy` is not directly used in this snippet but is a common companion to these libraries.
4.  **Data handling**: NaN filtering using `~t['column'].isnull()` is the correct pandas equivalent of MATLAB's `~isnan(t.column)`.
5.  **Plotting**: Th


✅ Conversion workflow completed!

❌ Error: No sessions found


### View and Save the Converted Python Code

The conversion is complete! The agent system has iteratively refined the code through validation cycles.

Let's extract and display the final Python code:

In [29]:
# Extract and save the converted Python code
output_filename = 'converted_matlab_to_python.py'

# Get the most recent session from the runner
try:
    # List all sessions for this user and app
    sessions_response = await converter_runner.session_service.list_sessions(
        user_id="default_user",
        app_name="MATLABtoPythonConverter"
    )
    
    if sessions_response.sessions and len(sessions_response.sessions) > 0:
        # Get the most recent session (first in list)
        session = await converter_runner.session_service.get_session(
            user_id="default_user",
            session_id=sessions_response.sessions[0].id
        )
        
        # Extract the python_code from the session state
        if 'python_code' in session.state:
            python_code = session.state['python_code']
            
            # Save to file
            with open(output_filename, 'w') as f:
                f.write(python_code)
            
            print("✅ Conversion complete!")
            print(f"📝 Python code saved to: {output_filename}")
            print(f"📂 File location: {os.path.abspath(output_filename)}")
            print("\n" + "="*60)
            print("The LoopAgent workflow:")
            print("1. ✓ Initial conversion from MATLAB to Python")
            print("2. ✓ Validation of Python code correctness")
            print("3. ✓ Iterative refinement (up to 3 cycles)")
            print("4. ✓ Exit when code meets all quality criteria")
            print("="*60)
            print("\n📄 Converted Python Code Preview:")
            print("-"*60)
            print(python_code)
            print("-"*60)
        else:
            print("❌ Error: Could not find 'python_code' in session state")
            print(f"Available state keys: {list(session.state.keys())}")
            print("\nAttempting alternative extraction...")
            
            # Try to find any state key that might contain the code
            for key, value in session.state.items():
                if isinstance(value, str) and len(value) > 50 and ('import' in value.lower() or 'pandas' in value.lower()):
                    print(f"\nFound code-like content in key '{key}':")
                    python_code = value
                    with open(output_filename, 'w') as f:
                        f.write(python_code)
                    print(f"✅ Saved to {output_filename}")
                    print("-"*60)
                    print(python_code[:1000] + "..." if len(python_code) > 1000 else python_code)
                    print("-"*60)
                    break
    else:
        print("❌ Error: No sessions found")
        print("Make sure you ran the conversion cell above first!")
        
except Exception as e:
    print(f"❌ Error retrieving session: {e}")
    print("\nTrying direct approach from runner...")
    print("Make sure the conversion cell completed successfully!")

❌ Error: No sessions found
Make sure you ran the conversion cell above first!


### Summary: Why LoopAgent for Code Conversion?

**Key Benefits:**

1. **Iterative Quality Improvement**: Code conversion rarely works perfectly on first attempt. The LoopAgent allows multiple refinement cycles.

2. **Automated Validation**: The validator agent checks for completeness, correctness, and best practices automatically.

3. **Deterministic Exit Condition**: The loop exits when code is approved or max iterations reached, preventing infinite loops.

4. **Separation of Concerns**: 
   - Converter focuses on translation
   - Validator focuses on correctness
   - Refiner focuses on improvements

5. **Traceable Process**: Each iteration's validation feedback and improvements are tracked in the agent's execution flow.

**Alternative Approaches Considered:**
- **Simple Agent**: Would do one-shot conversion without validation or refinement
- **Sequential Agent**: Would validate once but couldn't iterate for improvements
- **LoopAgent** ✓ **BEST CHOICE**: Enables iterative refinement until quality standards are met

This pattern is ideal for any code transformation, migration, or generation task where quality matters and initial attempts may need refinement.